In [ ]:
import json
import re
from src.metrics import (
    tokenize, cosine_similarity, completeness_score,
    hallucination_score, estimate_tokens, estimate_cost
)

def load_file_text(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()


def load_json_safe(path):
    with open(path, "r", encoding="utf-8") as f:
        raw = f.read()

    # Fix common JSON issues
    raw = raw.replace(",\n}", "\n}")
    raw = raw.replace(",\n]", "\n]")

    try:
        return json.loads(raw)
    except Exception as e:
        print("⚠️ JSON parsing failed, falling back to text-only mode")
        return None

def extract_last_turns_fallback(raw_text):
    user_msgs = re.findall(
        r'"role"\s*:\s*"User".*?"message"\s*:\s*"(.*?)"',
        raw_text,
        re.DOTALL
    )
    ai_msgs = re.findall(
        r'"role"\s*:\s*"AI/Chatbot".*?"message"\s*:\s*"(.*?)"',
        raw_text,
        re.DOTALL
    )

    user_q = user_msgs[-1] if user_msgs else ""
    ai_a = ai_msgs[-1] if ai_msgs else ""

    return user_q, ai_a



def aggregate_context(context_json):
    if context_json is None:
        return ""

    texts = []
    for item in context_json.get("data", {}).get("vector_data", []):
        text = item.get("text")
        if isinstance(text, str):
            texts.append(text)

    return " ".join(texts)


def evaluate(chat_path, context_path):
    chat_text = load_file_text(chat_path)
    context_text = load_file_text(context_path)

    try:
        chat_json = json.loads(chat_text)
        user_q, ai_a = extract_last_turns(chat_json)
    except Exception:
        print("⚠️ JSON parsing failed, using regex fallback")
        user_q, ai_a = extract_last_turns_fallback(chat_text)

    if not user_q or not ai_a:
        raise ValueError("Could not extract user/AI messages")

    tokens = estimate_tokens(ai_a)

    return {
        "relevance_to_user": cosine_similarity(tokenize(user_q), tokenize(ai_a)),
        "completeness": completeness_score(user_q, ai_a),
        "hallucination_risk": hallucination_score(ai_a, context_text),
        "tokens": tokens,
        "estimated_cost_usd": estimate_cost(tokens)
    }
